<a href="https://colab.research.google.com/github/kazishuvo22/BigMart_Sales_Prediction/blob/master/BigMart_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BigMart Sales prediction Dataset**

---

Find out the properties of a product, and store which impacts the sales of a product.

---




In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
import glob
%matplotlib inline


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_file="/content/drive/My Drive/cse445 dataset/Train.csv"
test_file="/content/drive/My Drive/cse445 dataset/Test.csv"
submission_file="/content/drive/My Drive/cse445 dataset/Submission.csv"

In [5]:
train_file, test_file, submission_file

('/content/drive/My Drive/cse445 dataset/Train.csv',
 '/content/drive/My Drive/cse445 dataset/Test.csv',
 '/content/drive/My Drive/cse445 dataset/Submission.csv')

In [6]:
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
submission = pd.read_csv(submission_file)

In [7]:
train

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [8]:
test

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3
...,...,...,...,...,...,...,...,...,...,...,...
5676,FDB58,10.500,Regular,0.013496,Snack Foods,141.3154,OUT046,1997,Small,Tier 1,Supermarket Type1
5677,FDD47,7.600,Regular,0.142991,Starchy Foods,169.1448,OUT018,2009,Medium,Tier 3,Supermarket Type2
5678,NCO17,10.000,Low Fat,0.073529,Health and Hygiene,118.7440,OUT045,2002,NaN,Tier 2,Supermarket Type1
5679,FDJ26,15.300,Regular,0.000000,Canned,214.6218,OUT017,2007,NaN,Tier 2,Supermarket Type1


In [9]:
submission

,Unnamed: 0,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,0,FDW58,OUT049,1636.244023
1,1,FDW14,OUT017,1409.355910
2,2,NCN55,OUT010,710.594286
3,3,FDQ58,OUT017,2355.184955
4,4,FDY38,OUT027,5857.916205
...,...,...,...,...
5676,5676,FDB58,OUT046,2125.984810
5677,5677,FDD47,OUT018,2602.671833
5678,5678,NCO17,OUT045,1832.451358
5679,5679,FDJ26,OUT017,3538.685188


## Combine test and train into one file

To explore data in any data science competetion, it is advisable to append test data to the train data. So, we will combine both train and test to carry out data visualization, feature engineering, one-hot encoding, and label encoding. Later we would split this combined data back to train and test datasets.








In [10]:
train['source']='train'
test['source']='test'
data = pd.concat([train, test],ignore_index=True)
print(train.shape, test.shape, data.shape)

(8523, 13) (5681, 12) (14204, 13)


train dataset has 8523 rows and 12 features and test has 5681 rows and 11 columns. train has 1 extra column which is the target variable. We will predict this target variable for the test dataset later.

In [11]:
data.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
source                        object
dtype: object

In [12]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train


##Numerical data summary:

In [13]:
data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


## Data Cleaning
Missing data can have a severe impact on building predictive models because the missing values might be contain some vital information which could help in making better predictions. So, it becomes imperative to carry out missing data imputation.



### Check missing values:

In [14]:
data.apply(lambda x: sum(x.isnull()))

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64


### Filling missing values


In [15]:
data['Item_Outlet_Sales'].value_counts()

958.7520     17
1342.2528    16
1845.5976    15
703.0848     15
1278.3360    14
             ..
3167.8764     1
2226.4352     1
1684.4740     1
1574.6170     1
6692.6216     1
Name: Item_Outlet_Sales, Length: 3493, dtype: int64

In [16]:
data.Item_Outlet_Sales = data.Item_Outlet_Sales.fillna(data.Item_Outlet_Sales.mean())

In [17]:
data.Item_Outlet_Sales 

0        3735.138000
1         443.422800
2        2097.270000
3         732.380000
4         994.705200
            ...     
14199    2181.288914
14200    2181.288914
14201    2181.288914
14202    2181.288914
14203    2181.288914
Name: Item_Outlet_Sales, Length: 14204, dtype: float64

In [18]:
data['Item_Weight'].value_counts()

17.600    135
12.150    127
10.500    123
13.650    115
11.800    113
         ... 
5.210       7
7.960       7
4.615       6
9.035       6
7.850       6
Name: Item_Weight, Length: 415, dtype: int64

In [19]:
data.Item_Weight = data.Item_Weight.fillna(data.Item_Weight.mean())
data.Item_Weight

0         9.30
1         5.92
2        17.50
3        19.20
4         8.93
         ...  
14199    10.50
14200     7.60
14201    10.00
14202    15.30
14203     9.50
Name: Item_Weight, Length: 14204, dtype: float64

In [20]:

data['Outlet_Size'].value_counts()

Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

In [21]:
data.Outlet_Size = data.Outlet_Size.fillna('Medium')
data.Outlet_Size

0        Medium
1        Medium
2        Medium
3        Medium
4          High
          ...  
14199     Small
14200    Medium
14201    Medium
14202    Medium
14203    Medium
Name: Outlet_Size, Length: 14204, dtype: object

In [22]:
#Again check missing values
data.apply(lambda x: sum(x.isnull()))

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
source                       0
dtype: int64

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                14204 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                14204 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  Item_Outlet_Sales          14204 non-null  float64
 12  source                     14204 non-null  object 
dtypes: float64(4), int64(1), object(8)
memory usag

### Item type combine:

In [24]:
data['Item_Identifier'].value_counts()


FDM60    10
FDR27    10
FDU26    10
NCH42    10
FDW01    10
         ..
NCL42     7
FDO33     7
FDS22     7
FDX49     7
NCW54     7
Name: Item_Identifier, Length: 1559, dtype: int64

In [25]:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})

In [26]:
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

## Numerical and One-Hot Coding of Categorical variables

In [27]:
#import library
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label = LabelEncoder()
#New variable for outlet
data['Outlet'] = label.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
label = LabelEncoder()
for i in var_mod:
    data[i] = label.fit_transform(data[i])

In [28]:
data

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Item_Type_Combined,Outlet
0,FDA15,9.30,1,0.016047,Dairy,249.8092,OUT049,1999,1,0,1,3735.138000,train,1,9
1,DRC01,5.92,2,0.019278,Soft Drinks,48.2692,OUT018,2009,1,2,2,443.422800,train,0,3
2,FDN15,17.50,1,0.016760,Meat,141.6180,OUT049,1999,1,0,1,2097.270000,train,1,9
3,FDX07,19.20,2,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,1,2,0,732.380000,train,1,0
4,NCD19,8.93,1,0.000000,Household,53.8614,OUT013,1987,0,2,1,994.705200,train,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14199,FDB58,10.50,2,0.013496,Snack Foods,141.3154,OUT046,1997,2,0,1,2181.288914,test,1,8
14200,FDD47,7.60,2,0.142991,Starchy Foods,169.1448,OUT018,2009,1,2,2,2181.288914,test,1,3
14201,NCO17,10.00,1,0.073529,Health and Hygiene,118.7440,OUT045,2002,1,1,1,2181.288914,test,2,7
14202,FDJ26,15.30,2,0.000000,Canned,214.6218,OUT017,2007,1,1,1,2181.288914,test,1,2


In [29]:
#One Hot Coding:
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Item_Type_Combined','Outlet'])

In [30]:
data

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Item_Outlet_Sales,source,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Item_Fat_Content_4,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Outlet_Type_0,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Item_Type_Combined_0,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.016047,Dairy,249.8092,OUT049,1999,3735.138000,train,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,OUT018,2009,443.422800,train,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,Meat,141.6180,OUT049,1999,2097.270000,train,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,732.380000,train,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.000000,Household,53.8614,OUT013,1987,994.705200,train,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14199,FDB58,10.50,0.013496,Snack Foods,141.3154,OUT046,1997,2181.288914,test,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
14200,FDD47,7.60,0.142991,Starchy Foods,169.1448,OUT018,2009,2181.288914,test,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0
14201,NCO17,10.00,0.073529,Health and Hygiene,118.7440,OUT045,2002,2181.288914,test,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
14202,FDJ26,15.30,0.000000,Canned,214.6218,OUT017,2007,2181.288914,test,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0


In [31]:
data.head(6)

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Item_Outlet_Sales,source,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Item_Fat_Content_4,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Outlet_Type_0,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Item_Type_Combined_0,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.300,0.016047,Dairy,249.8092,OUT049,1999,3735.1380,train,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.920,0.019278,Soft Drinks,48.2692,OUT018,2009,443.4228,train,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.500,0.016760,Meat,141.6180,OUT049,1999,2097.2700,train,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.200,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,732.3800,train,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
4,NCD19,8.930,0.000000,Household,53.8614,OUT013,1987,994.7052,train,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
5,FDP36,10.395,0.000000,Baking Goods,51.4008,OUT018,2009,556.6088,train,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0


In [32]:
data.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Item_Outlet_Sales            float64
source                        object
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Item_Fat_Content_2             uint8
Item_Fat_Content_3             uint8
Item_Fat_Content_4             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
O

##Data split and Exporting Data

In [33]:
#Drop the columns which have been converted to different types:
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

In [34]:
data.dtypes

Item_Identifier            object
Item_Weight               float64
Item_Visibility           float64
Item_MRP                  float64
Outlet_Identifier          object
Item_Outlet_Sales         float64
source                     object
Item_Fat_Content_0          uint8
Item_Fat_Content_1          uint8
Item_Fat_Content_2          uint8
Item_Fat_Content_3          uint8
Item_Fat_Content_4          uint8
Outlet_Location_Type_0      uint8
Outlet_Location_Type_1      uint8
Outlet_Location_Type_2      uint8
Outlet_Size_0               uint8
Outlet_Size_1               uint8
Outlet_Size_2               uint8
Outlet_Type_0               uint8
Outlet_Type_1               uint8
Outlet_Type_2               uint8
Outlet_Type_3               uint8
Item_Type_Combined_0        uint8
Item_Type_Combined_1        uint8
Item_Type_Combined_2        uint8
Outlet_0                    uint8
Outlet_1                    uint8
Outlet_2                    uint8
Outlet_3                    uint8
Outlet_4      

In [35]:
#Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

In [36]:
train.items

<bound method DataFrame.items of      Item_Identifier  Item_Weight  ...  Outlet_8  Outlet_9
0              FDA15        9.300  ...         0         1
1              DRC01        5.920  ...         0         0
2              FDN15       17.500  ...         0         1
3              FDX07       19.200  ...         0         0
4              NCD19        8.930  ...         0         0
...              ...          ...  ...       ...       ...
8518           FDF22        6.865  ...         0         0
8519           FDS36        8.380  ...         0         0
8520           NCJ29       10.600  ...         0         0
8521           FDN46        7.210  ...         0         0
8522           DRG01       14.800  ...         1         0

[8523 rows x 35 columns]>

In [37]:
test.items

<bound method DataFrame.items of       Item_Identifier  Item_Weight  ...  Outlet_8  Outlet_9
8523            FDW58    20.750000  ...         0         1
8524            FDW14     8.300000  ...         0         0
8525            NCN55    14.600000  ...         0         0
8526            FDQ58     7.315000  ...         0         0
8527            FDY38    12.792854  ...         0         0
...               ...          ...  ...       ...       ...
14199           FDB58    10.500000  ...         1         0
14200           FDD47     7.600000  ...         0         0
14201           NCO17    10.000000  ...         0         0
14202           FDJ26    15.300000  ...         0         0
14203           FDU37     9.500000  ...         0         0

[5681 rows x 35 columns]>

In [38]:
#Drop unnecessary columns:
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
#Export files as modified versions:
train.to_csv("/content/drive/My Drive/cse445 dataset/train_modified.csv",index=False)
test.to_csv("/content/drive/My Drive/cse445 dataset/test_modified.csv",index=False)

##Export modified Data

In [40]:
 #Reading modified data
 train_n= pd.read_csv("/content/drive/My Drive/cse445 dataset/train_modified.csv")
 test_n= pd.read_csv("/content/drive/My Drive/cse445 dataset/test_modified.csv")
 train_n, test_n

(     Item_Identifier  Item_Weight  ...  Outlet_8  Outlet_9
 0              FDA15        9.300  ...         0         1
 1              DRC01        5.920  ...         0         0
 2              FDN15       17.500  ...         0         1
 3              FDX07       19.200  ...         0         0
 4              NCD19        8.930  ...         0         0
 ...              ...          ...  ...       ...       ...
 8518           FDF22        6.865  ...         0         0
 8519           FDS36        8.380  ...         0         0
 8520           NCJ29       10.600  ...         0         0
 8521           FDN46        7.210  ...         0         0
 8522           DRG01       14.800  ...         1         0
 
 [8523 rows x 34 columns],
      Item_Identifier  Item_Weight  ...  Outlet_8  Outlet_9
 0              FDW58    20.750000  ...         0         1
 1              FDW14     8.300000  ...         0         0
 2              NCN55    14.600000  ...         0         0
 3         

In [41]:
train_n.head(4)

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Identifier,Item_Outlet_Sales,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Item_Fat_Content_4,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Outlet_Type_0,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Item_Type_Combined_0,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.016047,249.8092,OUT049,3735.1380,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,48.2692,OUT018,443.4228,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,141.6180,OUT049,2097.2700,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.000000,182.0950,OUT010,732.3800,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0


In [42]:
test_n.head(4)

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Identifier,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Item_Fat_Content_4,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Outlet_Type_0,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Item_Type_Combined_0,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDW58,20.750,0.007565,107.8622,OUT049,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,FDW14,8.300,0.038428,87.3198,OUT017,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2,NCN55,14.600,0.099575,241.7538,OUT010,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
3,FDQ58,7.315,0.015388,155.0340,OUT017,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0


##Model Building

In [43]:
X_train = train_n.drop(['Item_Outlet_Sales', 'Outlet_Identifier','Item_Identifier'], axis=1)
Y_train = train_n.Item_Outlet_Sales

X_train, Y_train

(      Item_Weight  Item_Visibility  Item_MRP  ...  Outlet_7  Outlet_8  Outlet_9
 0           9.300         0.016047  249.8092  ...         0         0         1
 1           5.920         0.019278   48.2692  ...         0         0         0
 2          17.500         0.016760  141.6180  ...         0         0         1
 3          19.200         0.000000  182.0950  ...         0         0         0
 4           8.930         0.000000   53.8614  ...         0         0         0
 ...           ...              ...       ...  ...       ...       ...       ...
 8518        6.865         0.056783  214.5218  ...         0         0         0
 8519        8.380         0.046982  108.1570  ...         1         0         0
 8520       10.600         0.035186   85.1224  ...         0         0         0
 8521        7.210         0.145221  103.1332  ...         0         0         0
 8522       14.800         0.044878   75.4670  ...         0         1         0
 
 [8523 rows x 31 columns],

In [44]:
X_test = test_n.drop(['Outlet_Identifier','Item_Identifier'], axis=1)

X_test

,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Item_Fat_Content_4,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Outlet_Type_0,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Item_Type_Combined_0,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,20.750000,0.007565,107.8622,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,8.300000,0.038428,87.3198,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2,14.600000,0.099575,241.7538,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
3,7.315000,0.015388,155.0340,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4,12.792854,0.118599,234.2300,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5676,10.500000,0.013496,141.3154,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
5677,7.600000,0.142991,169.1448,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0
5678,10.000000,0.073529,118.7440,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
5679,15.300000,0.000000,214.6218,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0


In [45]:
X_train.head(4)

,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Item_Fat_Content_4,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Outlet_Type_0,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Item_Type_Combined_0,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,9.30,0.016047,249.8092,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,5.92,0.019278,48.2692,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,17.50,0.016760,141.6180,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
3,19.20,0.000000,182.0950,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0


In [46]:
Y_train.head(4)

0    3735.1380
1     443.4228
2    2097.2700
3     732.3800
Name: Item_Outlet_Sales, dtype: float64

In [47]:
X_test.head(4)

,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Item_Fat_Content_4,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Outlet_Type_0,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Item_Type_Combined_0,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,20.750,0.007565,107.8622,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,8.300,0.038428,87.3198,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2,14.600,0.099575,241.7538,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
3,7.315,0.015388,155.0340,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0


###Linear Regression Model:

In [48]:
# Fitting Multiple Linear Regression to the training set

from sklearn.linear_model import LinearRegression

r = LinearRegression()
r.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [49]:
# Predicting the test set results
y_predict = r.predict(X_test)

y_predict

array([1848.53604783, 1472.81670435, 1875.65285894, ..., 1809.18796433,
       3565.6645235 , 1267.46171871])

In [50]:
# Measuring Accuracy
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn import metrics


In [51]:
lr_accuracy = round(r.score(X_train,Y_train) *100 , 2)
lr_accuracy 

56.36

In [52]:
r2_score(Y_train, r.predict(X_train))

0.563589277727048

In [53]:
#Perform cross-validation
cv_score = cross_val_score(r, X_train,Y_train, cv=10, scoring='neg_mean_squared_error')

In [54]:
print(cv_score)
print(np.sqrt(np.abs(cv_score)))

[-1355870.39384577 -1297468.79646036 -1261250.73154846 -1232241.49900679
 -1253904.31889722 -1228218.74729252 -1241687.11113408 -1290746.34014763
 -1296451.4260274  -1309155.10757897]
[1164.41847883 1139.06487807 1123.05419796 1110.06373646 1119.77869193
 1108.25030895 1114.31015033 1136.11017958 1138.61820907 1144.18316173]


In [55]:
print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(Y_train, r.predict(X_train))))

RMSE : 1127


In [56]:
submission = pd.DataFrame({
'Item_Identifier':test_n['Item_Identifier'],
'Outlet_Identifier':test_n['Outlet_Identifier'],
'Item_Outlet_Sales': y_predict
},columns=['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales'])

In [57]:
submission.to_csv('/content/drive/My Drive/cse445 dataset/submission_linearReg.csv',index=False)

###Random Forest Model:

In [58]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100,max_depth=6, min_samples_leaf=50,n_jobs=4)
regressor.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=50,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=4, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [59]:
# Predicting the test set results
y_predict = regressor.predict(X_test)
y_predict

array([1658.67808682, 1356.54766146,  594.71089599, ..., 1944.00335895,
       3669.26596267, 1282.94430342])

In [60]:
# Measuring Accuracy
rf_accuracy = round(regressor.score(X_train,Y_train)*100,2)
rf_accuracy

61.28

In [61]:
r2_score(Y_train, regressor.predict(X_train))

0.6128153780268656

In [62]:
#Perform cross-validation
cv_score = cross_val_score(regressor, X_train,Y_train, cv=10, scoring='neg_mean_squared_error')

In [63]:
print(cv_score)
print(np.sqrt(np.abs(cv_score)))

[-1223017.85973479 -1182235.44988771 -1188625.60705611 -1120303.09963298
 -1181477.69543724 -1128190.55839773 -1103497.95430965 -1171090.73676664
 -1127114.52433652 -1228016.40543166]
[1105.90137885 1087.30651147 1090.24107749 1058.44371586 1086.95800077
 1062.16315056 1050.47510885 1082.16945843 1061.65650016 1108.15901631]


In [64]:
print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(Y_train, regressor.predict(X_train))))

RMSE : 1062


In [ ]:
submission = pd.DataFrame({
'Item_Identifier':test_n['Item_Identifier'],
'Outlet_Identifier':test_n['Outlet_Identifier'],
'Item_Outlet_Sales': y_predict
},columns=['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales'])

In [ ]:
submission.to_csv('/content/drive/My Drive/cse445 dataset/submission_RandomForest.csv',index=False)

###XgBoost Model

In [ ]:
from xgboost import XGBClassifier
import pickle
from matplotlib import pyplot

In [ ]:
regressor_xg = XGBClassifier(colsample_bytree=0.3,learning_rate=0.1,max_depth=5,n_estimators=100,reg_alpha=0.75,reg_lambda=0.45,subsample=0.6,seed=42)

In [ ]:
regressor_xg.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
              silent=None, subsample=0.6, verbosity=1)

In [ ]:
# save model to file
pickle.dump(regressor_xg, open("/content/drive/My Drive/cse445 dataset/save_model/xgb_bigMart.pickle.dat", "wb"))

In [ ]:
# Predicting the test set results
y_predict = regressor_xg.predict(X_test)
y_predict

array([1342.2528,  703.0848, 1426.1436, ..., 1540.6612, 2636.568 ,
        703.0848])

In [ ]:
print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(Y_train, regressor_xg.predict(X_train))))

RMSE : 1470


In [ ]:
submission = pd.DataFrame({
'Item_Identifier':test_n['Item_Identifier'],
'Outlet_Identifier':test_n['Outlet_Identifier'],
'Item_Outlet_Sales': y_predict
},columns=['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales'])

In [ ]:
submission.to_csv('/content/drive/My Drive/cse445 dataset/submission_Xgboost.csv',index=False)

In [ ]:
#Perform cross-validation
cv_score = cross_val_score(regressor_xg, X_train,Y_train, cv=10, scoring='neg_mean_squared_error')

In [ ]:
print(cv_score)
print(np.sqrt(np.abs(cv_score)))

In [ ]:
# Measuring Accuracy
xgb_accuracy = round(regressor_xg.score(X_train,Y_train)*100,2)
xgb_accuracy

In [ ]:
r2_score(Y_train, regressor_xg.predict(X_train))